### IMPORT CÁC THƯ VIỆN CẦN THIẾT:
- MATPLOTLIB: DÙNG ĐỂ HIỂN THỊ DỮ LIỆU
- NUMPY: THƯ VIỆN ĐẠI SỐ TUYẾN TÍNH, DÙNG ĐỂ XỬ LÝ TOÁN HỌC VỚI MA TRẬN DỮ LIỆU
- PANDAS: DÙNG ĐỂ ĐỌC FILE VÀ THAO TÁC TRÊN FILE DỮ LIỆU


In [130]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 


### MỞ FILE SỐ LIỆU, TRỘN CÁC DÒNG VỚI NHAU MỘT CÁCH NGẪU NHIÊN

In [131]:
X_train = pd.read_excel("ANNrauma.xlsx", sheet_name= "X").values
X_test = pd.read_excel("ANNrauma.xlsx", sheet_name= "Xtest").values
Y_train = pd.read_excel("ANNrauma.xlsx", sheet_name= "Y").values
Y_test = pd.read_excel("ANNrauma.xlsx", sheet_name= "Ytest").values

s = np.arange(0, len(X_train), 1)
np.random.shuffle(s)
X_train = X_train[s]
Y_train = Y_train[s]
s = np.arange(0, len(X_test), 1)
X_test = X_test[s]
Y_test = Y_test[s]


##### CHIA DỮ LIỆU TRAIN THÀNH CÁC GÓI NHỎ ĐỂ TRÁNH BỊ OVERFITTING KHI LUYỆN MẠNG

In [132]:
Y_chunks = np.array_split(Y_train, len(Y_train) / 18)
X_chunks = np.array_split(X_train, len(X_train) / 18)

### KHAI BÁO CÁC HÀM KÍCH HOẠT THƯỜNG SỬ DỤNG

In [133]:
def ReLU(input):
    return np.maximum(input, 0)
def leaky_ReLU(input):
    return np.maximum(0.07 * input, input)
def Linear(input):
    return input
def Sigmoid(input):
    return 1 / (1 + np.exp(-input))
def Swish(input):
    return input * Sigmoid(input)

### KHAI BÁO ĐẠO HÀM CỦA CÁC HOẠT KÍCH HOẠT THEO DỮ LIỆU ĐẦU VÀO

In [134]:
def ReLU_de(input):
    return (input > 0) * 1
def leaky_ReLU_de(input):
    return np.where(input > 0, 1, 0.07) 
def Linear_de(input):
    return 1
def Sigmoid_de(input):
    return np.exp(input)/ np.square(np.exp(input) + 1)
def Swish_de(input):
    return Swish(input) + Sigmoid(input)*(1 - Swish(input))

##### HÀM NHẬN MỘT HÀM KHÁC VÀ TRẢ VỀ ĐẠO HÀM TƯƠNG ỨNG

In [135]:
def deriative_func(func):
    if func.__name__ == "ReLU":
        return (ReLU_de)
    if func.__name__ == "leaky_ReLU":
        return (leaky_ReLU_de)
    if func.__name__ == "Linear":
        return (Linear_de)
    if func.__name__ == "Sigmoid":
        return (Sigmoid_de)
    if func.__name__ == "Swish":
        return (Swish_de)

### KHAI BÁO CLASS LAYER CỦA MẠNG NỞ RON NHÂN TẠO

In [136]:
class Layer:
    def __init__(self, dim, act_func, index = 0, isoutputlayer = False):
        self.isoutputlayer = isoutputlayer
        self.dim = dim
        self.weight = 0.01 * (2 * np.random.random(dim) - 1)
        self.act_func = act_func
        self.de_act_func = deriative_func(act_func)
        self.delta = None
        self.input = None
        self.d_input = None
        self.output = None
    
    def forward(self, input_data):
        self.input =  np.dot(input_data, self.weight) 
        self.d_input = self.de_act_func(self.input)
        self.output = self.act_func(self.input)
        return self.output
    
    def backward(self, right_layer, target):
        if self.isoutputlayer == True:
            self.delta = np.atleast_2d(target - self.output) * self.d_input
        else:
            self.delta = np.dot(right_layer.delta, right_layer.weight.T) * self.d_input
        return self.delta

    def update(self, left_layer_output, learning_rate):
        self.weight -= learning_rate * np.atleast_2d(np.dot(left_layer_output.T, self.delta))  
    

### KHAI BÁO CLASS (BẢN VẼ) MẠNG NƠ RON NHÂN TẠO

In [137]:
class Neural_Network:
    def __init__(self, n_nodes_list, hiddenlayers_actfunc):
        self.n_layers = len(n_nodes_list) - 1
        self.layers = []
        for i in range (1, len(n_nodes_list) - 1):
            self.layers.append(Layer(dim = (n_nodes_list[i - 1] + 1, n_nodes_list[i] + 1), act_func=hiddenlayers_actfunc, index = i - 1))
        self.layers.append(Layer(dim = (n_nodes_list[i] + 1, n_nodes_list[i + 1]), act_func=Linear, index = i + 1, isoutputlayer=True))
    def fit(self, input_matrix, target, learning_rate, epochs):
        input_matrix = np.concatenate((input_matrix, np.atleast_2d(np.ones(input_matrix.shape[0])).T), axis=1)
        for epoch in range(epochs):
            self.layers[0].forward(input_matrix)
            for i in range(1, self.n_layers):
                self.layers[i].forward(self.layers[i - 1].output)
            self.layers[-1].delta = self.layers[-1].output - target
            for i in range (self.n_layers - 2, -1 , -1):
                self.layers[i].backward(self.layers[i + 1], target)
            self.layers[0].update(input_matrix, learning_rate)
            for i in range (1, self.n_layers):
                self.layers[i].update(self.layers[i - 1].output, learning_rate)    

    def predict(self, input_matrix):
        input_matrix = np.concatenate((input_matrix, np.atleast_2d(np.ones(input_matrix.shape[0])).T), axis=1)
        self.layers[0].forward(input_matrix)
        for i in range(1, self.n_layers):
            self.layers[i].forward(self.layers[i - 1].output)
        return self.layers[-1].output

##### TẠO RA MỘT MODEL MẠNG NƠRON ĐỂ SỬ DỤNG, SAU ĐÓ LẦN LƯỢT SỬ DỤNG CÁC GÓI DỮ LIỆU ĐÃ CHIA ĐỂ LUYỆN MODEL NÀY

In [138]:
model = Neural_Network([X_train.shape[1], 50, 40, 30,  Y_train.shape[1]], hiddenlayers_actfunc = leaky_ReLU)
for i in range (len(X_chunks)):
    model.fit(X_chunks[i], Y_chunks[i], learning_rate=0.01, epochs=50000)

##### SỬ DỤNG MODEL ĐỂ DỤ ĐOÁN VỚI TẬP DỮ LIỆU KIỂM TRA

In [139]:
Y_pred = model.predict(X_test)
print(Y_pred)

[[0.04852521 0.07496778]
 [0.02995131 0.04247113]
 [0.06342148 0.09926856]
 [0.05043883 0.07707804]
 [0.11076734 0.17989936]
 [0.12100926 0.19888383]
 [0.04009547 0.06059014]
 [0.00977071 0.01100436]
 [0.01265698 0.01457494]
 [0.0968542  0.15711719]
 [0.01187737 0.01390789]
 [0.02201912 0.03082387]
 [0.08456134 0.13655284]
 [0.03831796 0.05742863]
 [0.03222517 0.04734664]
 [0.02840511 0.04111034]]
